In [ ]:
## FUNCIONES

In [3]:
import sys
import os
import csv

sys.path.append(os.path.abspath("../Data"))

def leer_archivo(ruta_archivo):
    with open(ruta_archivo, encoding='utf-8') as archivo_csv:
        reader = csv.reader(archivo_csv, delimiter=";")
        return list(reader)

def procesar_archivos(ruta, tipo="hogar"):
    datos_unificados = []
    cabecera = None

    for nombre_archivo in os.listdir(ruta):
        if tipo in nombre_archivo and nombre_archivo.endswith(".txt"):
            ruta_archivo = os.path.join(ruta, nombre_archivo)
            filas = leer_archivo(ruta_archivo)

            if not filas:
                continue

            if cabecera is None:
                cabecera = filas[0]
                datos_unificados.extend(filas[1:]) 
            else:
                if filas[0] == cabecera:
                    datos_unificados.extend(filas[1:])
                else:
                    print(f"⚠️ Cabecera diferente en: {nombre_archivo}")

    return [cabecera] + datos_unificados

def guardar_en_csv(datos, ruta_destino, nombre_archivo="hogares_unificados.csv"):
    # Crear la ruta completa del archivo
    ruta_completa = os.path.join(ruta_destino, nombre_archivo)

    # Asegurarse de que la carpeta de destino exista (solo la carpeta, no el archivo)
    os.makedirs(ruta_destino, exist_ok=True)

    # Guardar los datos en el archivo CSV
    with open(ruta_completa, mode='w', encoding='utf-8', newline='') as archivo_csv:
        writer = csv.writer(archivo_csv, delimiter=";")
        writer.writerows(datos)
    
    print(f"✅ Archivo CSV guardado en: {ruta_completa}")


# Actividades Previas

## Parte 1 - Con el archivo individual

In [58]:
ruta = "../Data/data_raw"
nombre_archivo="usu_individual_T324.txt"
ruta_archivo = os.path.join(ruta, nombre_archivo)
with open(ruta_archivo, encoding='utf-8') as archivo_csv:
    reader = csv.reader(archivo_csv, delimiter=";")
    encabezado=next(reader)
    print() 

    #acumulador
    masculino=0
    femenino=0
    mayores_edad_secu_completo=0

    #mis filas
    genero=11
    pondero=9
    educacion=encabezado.index("NIVEL_ED")
    edad=encabezado.index("CH06")
    encabezado.append("CH04_str")
    
    for fila in reader:
    
    #GENERO 
        if int(fila[genero])==1:
            masculino=masculino+int(fila[pondero])
            fila.append('Masculino')
                     
        else:
            femenino=femenino+int(fila[pondero])
            fila.append('Femenino')
        
    #PERSONAS MAYORES DE EDAD Y ESTUDIOS SECUNDARIOS
        if (int(fila[educacion]) in (4,5,6)) and int(fila[edad])>=18:
            mayores_edad_secu_completo=mayores_edad_secu_completo+int(fila[pondero])
   
#CANTIDAD DE HOMBRES Y MUJERES EPH
print(f"Total Hombres: {masculino}")
print(f"Total Mujeres: {femenino}")
print("-"*20)
print(f"Total de mayores de edad con secundario completo: {mayores_edad_secu_completo}")
print("-"*20)


Total Hombres: 14471801
Total Mujeres: 15248344
--------------------
Total de mayores de edad con secundario completo: 14148491
--------------------


## Parte 2 - Con el archivo hogares

In [6]:
ruta = "../Data/data_raw"
nombre_archivo="usu_hogar_T324.txt"
ruta_archivo = os.path.join(ruta, nombre_archivo)
with open(ruta_archivo, encoding='utf-8') as archivo_csv:
    reader = csv.reader(archivo_csv, delimiter=";")
    encabezado=next(reader)

    #Filas indices
    propietario=encabezado.index("II7")
    baño=encabezado.index("IV8")
    cant_hab=encabezado.index("IX_TOT")
    aglomerado=encabezado.index('AGLOMERADO')
    pondera=encabezado.index('PONDERA')

#inicializo variabales
    prop=0
    total_viviendas=0
    aglomerado_contador={}
    
#Informen el porcentaje de viviendas que son ocupadas por el/la propietario/a de la vivienda y del terreno.
    for fila in reader:
        if fila[propietario]=='1':
            prop+=int(fila[pondera])
        total_viviendas+=int(fila[pondera])
    
# Informen el aglomerado con mayor y menor cantidad de viviendas con más de 2 ocupantes que no posean baño.
        if fila[baño]=='2' and  int(fila[cant_hab])>2:
            if fila[aglomerado] not in aglomerado_contador:
                aglomerado_contador[fila[aglomerado]]=int(fila[pondera])
            else:
                aglomerado_contador[fila[aglomerado]]+=int(fila[pondera])

print(f"El porcentaje de viviendas ocupadas por el/la propietario/a del terreno y la vivienda es: {round((prop / total_viviendas) * 100, 2)}%")

aglo_max = max(aglomerado_contador, key=lambda x: aglomerado_contador[x])
aglo_min = min(aglomerado_contador, key=lambda x: aglomerado_contador[x])

print(f"Aglomerado con mayor cantidad de viviendas sin baño y más de dos ocupantes: {aglo_max} con un total de ({aglomerado_contador[aglo_max]}) hogares")
print(f"Aglomerado con menor cantidad de viviendas sin baño y más de dos ocupantes: {aglo_min} con un total de ({aglomerado_contador[aglo_min]}) hogares")

El porcentaje de viviendas ocupadas por el/la propietario/a del terreno y la vivienda es: 60.0%
Aglomerado con mayor cantidad de viviendas sin baño y más de dos ocupantes: 33 con un total de (8469) hogares
Aglomerado con menor cantidad de viviendas sin baño y más de dos ocupantes: 6 con un total de (198) hogares


## TAREAS

1. Se debe definir una carpeta, cuyo path debe ser confi gurable vía una variable, donde se almacenarán el conjunto de archivos de “individuos” y “hogares”

In [ ]:
ruta_destino = "../Data/data_clean"

2. Se debe generar una sección de código dentro del notebook que busque en la mencionada carpeta cada uno de los archivos “individuos” y “hogares” y los una. Es decir, si en la carpeta tenemos tres archivos de individuos (año 2023 trimestre 1, año 2023 trimestre 2, año 2024 trimestre 1) se debe leer la información de cada uno y generar un dataset que contenga toda la información.

In [ ]:
ruta = "../Data/data_raw"

#Proceso Dataset individuos 
individual=procesar_archivos(ruta, tipo="individual")
ruta_destino = "../Data/data_clean"
guardar_en_csv(individual,ruta_destino,"individuales_unificado")

#Proceso  Dataset hogares 
hogares=procesar_archivos(ruta, tipo="hogar")

guardar_en_csv(hogares,ruta_destino,"hogares_unificado")

✅ Archivo CSV guardado en: ../Data/data_clean\individuales_unificado
✅ Archivo CSV guardado en: ../Data/data_clean\hogares_unificado


## ABRO UN ARCHIVO

3. Se debe traducir los valores CH04 numéricos a "Masculino" y "Femenino" según corresponda. El resultado se debe almacenar en una nueva columna llamada CH04_str.

# Informacion individuos

In [30]:
ruta = "../Data/data_raw"
nombre_archivo="usu_individual_T324.txt"
ruta_archivo = os.path.join(ruta, nombre_archivo)
with open(ruta_archivo, encoding='utf-8') as archivo_csv:
    reader = csv.reader(archivo_csv, delimiter=";")
    encabezado=next(reader)
    print() # o NIVEL_ED" CH12

    #mis filas
    genero=11
    pondero=9
    educacion=encabezado.index("NIVEL_ED")
    edad=encabezado.index("CH06")
    
    encabezado.append("CH04_str")
    
    for fila in reader:
    
    #GENERO 
        if int(fila[genero])==1:
            fila.append('Masculino')
                     
        else:
            fila.append('Femenino')        
            
                     
        




Informen el aglomerado con mayor y menor cantidad de viviendas con más de 2 ocupantes que no posean baño.

